In [30]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import shap
import sklearn.metrics as metrics

In [31]:
data_file = f"data/5_mr_50_cond/simulated_noNoise.txt"  #5 or 40 or 100

data = pd.read_csv(data_file, sep="\t", header=0)

N_genes = 100  # total no. of genes
N_TFs = N_genes

In [32]:
n_estimators=1000  # number of trees in the forest (as per GENIE paper)
criterion='squared_error'  # variance reduction equivalent
max_features = int(np.sqrt(N_genes-1)) # max no. of features to use in each tree (as per GENIE paper)
random_state = 42  # for reproducibility

In [33]:
# Normalize Expression data to unit-variance
data_n = StandardScaler(with_mean=False).fit_transform(data.to_numpy())

# Initialize matrices
W = np.zeros(shape=(N_genes,N_TFs))
Fscores = np.zeros(shape=(N_genes,))

for j in np.arange(0,N_genes):
    # read TF and gene expression data X and Gj
    X, Gj= data_n[:,:N_TFs], data_n[:,N_genes+j]
    
    # fit an RF model to predict gene expression from TF
    M_rf = RandomForestRegressor(criterion=criterion, n_estimators=n_estimators, max_features=max_features, random_state=random_state).fit(X,Gj)

    # train score
    Fscores[j] = M_rf.score(X,Gj)

    # Get the weights for all edges connecting TFs to gene j
    W[j,:] = M_rf.feature_importances_

    # # # look at feature importance based on SHAP values
    # # explainer = shap.TreeExplainer(M_rf)
    # # shap_values = explainer(X)